<a href="https://colab.research.google.com/github/vamosya/data_notebooks_python/blob/main/GarminActivitiesToCsv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install garminconnect

     |████████████████████████████████| 96 kB 4.0 MB/s 
     |████████████████████████████████| 54 kB 2.9 MB/s 
  Created wheel for garminconnect: filename=garminconnect-0.1.23-py3-none-any.whl size=7512 sha256=02fcf633a82769d357aa17eb4037eb2e387a4f04fb7d9a6d411f931f633db679
  Stored in directory: /root/.cache/pip/wheels/61/00/19/c3ce8b9a2f6d6d3582b70987431c358d1cf90f07644c0e2350
Successfully built garminconnect


In [2]:
pip install reverse_geocoder

     |████████████████████████████████| 2.2 MB 8.2 MB/s 
  Created wheel for reverse-geocoder: filename=reverse_geocoder-1.5.1-py3-none-any.whl size=2268087 sha256=dfbd29079b62973953dccf08f1fcf474a03b8843f4a5f997cb5f397b2d98ef25
  Stored in directory: /root/.cache/pip/wheels/34/6e/70/5423639428a2cac8ea7eb467214a4254b549b381f306a9c790
Successfully built reverse-geocoder


In [3]:
# Librerias necesarias
from garminconnect import Garmin
import logging
from getpass import getpass
import pandas as pd
import reverse_geocoder as rg
import warnings
warnings.filterwarnings("ignore")

In [4]:
# Conexión a Garmin Connect
paswd = getpass()
client = Garmin("cperaltap@gmail.com", paswd)
client.login()
print("Hola {}! Estás dentro!!".format(client.get_full_name()))

··········
Hola Txiki! Estás dentro!!


In [5]:
# Bajar actividades
activities = client.get_activities(0,2000) # tope de 2000. Se puede modificar

# guardo en un dataframe de Pandas
activities = pd.DataFrame(activities)

# Muestro id de la primera actividad
activities['activityId'][0]

7707003773

In [6]:
# Extraer de ActivityType el deporte y añadirlo a columna
sports = []
for i in range(len(activities)):
  sports.append(activities['activityType'][i]['typeKey'])
activities['sport'] = sports

# Muestro los deportes encontrados
activities['sport'].unique()

array(['running', 'hiking', 'mountain_biking', 'cycling', 'paddling',
       'stand_up_paddleboarding', 'open_water_swimming', 'inline_skating',
       'road_biking', 'rowing', 'strength_training', 'indoor_cycling',
       'treadmill_running', 'other', 'indoor_cardio', 'lap_swimming',
       'skating_ws', 'swimming', 'walking', 'boating', 'virtual_run',
       'elliptical', 'recumbent_cycling', 'rock_climbing',
       'indoor_rowing', 'fitness_equipment', 'tennis', 'casual_walking'],
      dtype=object)

In [7]:
# Redondeo para que haya menos casos diferentes
activities = activities.round({'startLatitude': 3})
activities = activities.round({'startLongitude': 3})

# Paso a string para trabajar mejor con los nan
activities['startLatitude'] = activities['startLatitude'].astype(str)
activities['startLongitude'] = activities['startLongitude'].astype(str)

# Sustituyo nan por 0.0
activities['startLatitude'].replace({"nan": "0.0"}, inplace=True)
activities['startLongitude'].replace({"nan": "0.0"}, inplace=True)

# Reseteo el index para facilitar iteración
activities.reset_index(drop=True, inplace=True)
activities.activityId

0       7707003773
1       7697323505
2       7687873716
3       7673681476
4       7660812921
           ...    
1122     853617665
1123     853617810
1124     853617944
1125     853618102
1126    2458506886
Name: activityId, Length: 1127, dtype: int64

In [8]:
# Creo la tupla necesaria para buscar después
activities['tupleLocation'] = activities[["startLatitude","startLongitude"]].apply(tuple, axis=1)
activities['tupleLocation'].value_counts()

(40.562, -3.622)    426
(0.0, 0.0)          313
(40.236, -4.385)     51
(40.566, -3.629)     20
(40.563, -3.621)     19
                   ... 
(40.475, -3.336)      1
(36.303, -6.118)      1
(38.933, -0.096)      1
(40.255, -4.37)       1
(40.56, -3.621)       1
Name: tupleLocation, Length: 166, dtype: int64

In [9]:
# Creo lista de ubicaciones únicas para hacer cache
cacheLocs = []
for i in range(len(activities)):
  cacheLocs.append(activities['tupleLocation'][i])
cacheLocs = list(set(cacheLocs))

# Creo el diccionario con estas ubicaciones únicas y su resultado.
tempLocs = {}
for i in range(len(cacheLocs)):
  tempLocs[cacheLocs[i]] = rg.search(cacheLocs[i])
tempLocs

# Inicializo las columnas nuevas
activities['cityLoc'] = ""
activities['provinceLoc'] = ""
activities['regionLoc'] = ""
activities['countryLoc'] = ""

# Itero para ir añadiendo los resultados a las columnas
for i in range(len(activities)):
  coordinates = activities['tupleLocation'][i]
  activities['cityLoc'][i] = tempLocs[coordinates][0]['name']
  activities['provinceLoc'][i] = tempLocs[coordinates][0]['admin1']
  activities['regionLoc'][i] = tempLocs[coordinates][0]['admin2']
  activities['countryLoc'][i] = tempLocs[coordinates][0]['cc']

# Muestro las regiones únicas donde hay actividades
activities.regionLoc.unique()

Loading formatted geocoded file...


array(['Provincia de Madrid', 'Provincia de Huelva', 'Silves',
       'Albufeira', 'Province of Toledo', 'Provincia de Valencia', '',
       'Provincia de Alicante', 'Provincia de Castello',
       'Provincia de Almeria', 'Provincia de Guadalajara',
       'Provincia de Segovia', 'Provincia de Cadiz',
       'Provincia de Burgos', 'Greater London', 'Murcia'], dtype=object)

In [10]:
# Creo el csv con las actividades descargadas
activities.to_csv('activities.csv')